In [1]:
import py2neo
import tqdm
import pandas as pd

from tqdm import tqdm
from py2neo import Graph,Node,Relationship,NodeMatcher

In [2]:
honglouGraph = Graph('http://localhost:7474',user = 'neo4j', password = '123')

In [3]:
nodeDF = pd.read_csv('/usr/local/codePro/bookCode/graph-algorithms-code/data/transport-nodes.csv')

In [4]:
relationDF = pd.read_csv('/usr/local/codePro/bookCode/graph-algorithms-code/data/transport-relationships.csv')

In [5]:
nodeDF.head()

,id,latitude,longitude,population
0,Amsterdam,52.379189,4.899431,821752
1,Utrecht,52.092876,5.104480,334176
2,Den Haag,52.078663,4.288788,514861
3,Immingham,53.612390,-0.222190,9642
4,Doncaster,53.522850,-1.131160,302400


In [6]:
relationDF.head()

,src,dst,relationship,cost
0,Amsterdam,Utrecht,EROAD,46
1,Amsterdam,Den Haag,EROAD,59
2,Den Haag,Rotterdam,EROAD,26
3,Amsterdam,Immingham,EROAD,369
4,Immingham,Doncaster,EROAD,74


In [7]:
for i, row in tqdm(nodeDF.iterrows()):
    
    transport_node = Node("transport_node",name=row[nodeDF.columns[0]])
    
    honglouGraph.merge(transport_node, "transport_node", "name")
    
    for j in range(len(list(row))-1):
        propertyKey = nodeDF.columns[j+1]
        propertyValue = row[nodeDF.columns[j+1]]
        transport_node.update({propertyKey:propertyValue})
        
    propertyKey = None
    propertyValue = None
        
    honglouGraph.push(transport_node)
    

12it [00:00, 88.09it/s]


In [8]:
for i, row in tqdm(relationDF.iterrows()):
    
    startNodeName, endNodeName, relationType = [row[j] for j in range(3)]
    startNode = honglouGraph.nodes.match("transport_node", name = startNodeName).first()
    endNode = honglouGraph.nodes.match("transport_node", name = endNodeName).first()
    transport_relationship = Relationship(startNode,relationType,endNode)
    
    honglouGraph.create(transport_relationship)
    
    for j in row.index[3:]:
        transport_relationship.update({j:row[j]})
    
    honglouGraph.push(transport_relationship)
    

15it [00:00, 48.77it/s]
